# The Anatomy of a Model 
## An Introduction to JuMP for Algebraic Modeling

_**[Power Systems Optimization](https://github.com/east-winds/power-systems-optimization)**_

_by Jesse D. Jenkins and Michael R. Davidson (last updated: September 4, 2020)_

This notebook will provide a basic introduction to [algebraic modeling](https://en.wikipedia.org/wiki/Algebraic_modeling_language) for [constrained optimization](https://en.wikipedia.org/wiki/Constrained_optimization) problems using the [Julia](https://julialang.org/) language and [JuMP](http://jump.dev/). The notebook will walk through the main components of an optimization model (decision variables, constraints, expressions, sets, objective function), show how to interface with a solver to find solutions to the model, and how to read solutions outputs back into Julia [DataFrames](https://juliadata.github.io/DataFrames.jl/stable/) and/or [write outputs to .csv files](https://juliadata.github.io/CSV.jl/stable/) for post-processing.

For additional resources, see the collection of notebooks provided at [JuMPTutorials.jl](https://github.com/jump-dev/JuMPTutorials.jl).

<img src="img/jump_logo.png" style="width: 450px; height: auto" align="left">

### What is JuMP?

[JuMP](http://jump.dev/) ("Julia for Mathematical Programming") is an open-source modeling language and supporting packages for mathematical optimization/algebraic programming in the [Julia](https://julialang.org/) high performance scientific programming language. 

JuMP makes it easy to formulate and solve linear programming, semidefinite programming, integer programming, convex optimization, constrained nonlinear optimization, and related classes of optimization problems, using easy-to-read code with syntax that closely matches the mathematical formulation of optimization problems. 

These problems can then be solved via a variety of open source or commercial optimization solvers, using Julia packages for interfacing with each solver. Thus, the definition of an algebraic model for a constrained optimization problem in JuMP is independent of the choice of solver. 

In this way, JuMP is similar to [AMPL](https://en.wikipedia.org/wiki/AMPL), [AIIMS](https://en.wikipedia.org/wiki/AIMMS) or [GAMS](https://en.wikipedia.org/wiki/General_Algebraic_Modeling_System) -- all specialized languages built for algebraic modeling. However, JuMP offers the advantage of being embeded within a fully functioning, high-level programming language (Julia). In this way, it is similar to the [Pyomo](http://www.pyomo.org/) package for Python, but offers the higher computational performance of Julia (and less verbose syntax). (For a brief [comparison between Julia and Python](https://devathon.com/blog/julia-vs-python-which-programming-language-is-better/), see here).

### Installing JuMP

JuMP is a package for Julia. From Julia, JuMP is installed by using the built-in package manager. [See this guide to installing and setting up Julia and JuMP](http://bit.ly/JuliaSetupGuide) if you've not yet done so. But very briefly, one installs JuMP with the following code:

In [80]:
# Note, you don't need to run this block if JuMP is already installed
import Pkg
Pkg.add("JuMP")

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


### Loading Julia packages

You only need to install JuMP once, but each time you start up Julia, you'll need to load the JuMP package and any other supporting packages you want to make use of. Here we'll load JuMP, [Cbc.jl](https://github.com/jump-dev/Clp.jl), the interface for the open source [COIN-OR Linear Programming solver](https://projects.coin-or.org/Clp).

In [81]:
# Note, you don't need to run this block if Clp package is already installed
import Pkg
Pkg.add("Clp")

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [7]:
# Load JuMP and Clp packages for use in this notebook
using JuMP
using Clp

## A factory example

<img src="img/factory.png" style="width: 800px; height: auto" align="left">

Let's start by recalling our simple factory optimization example from the [Constrained Optimization notebook](Notebooks/Constrained-Optimization.ipynb), which seeks to maximize total revenue as a function of how many widgets of type A (decision variable $x$) and type B (decision variable $y$) to produce:

$$
\begin{align}
& \max 150 x + 175 y &\text{Objective function (revenue)}\\ 
& \text{s.t.} & \\
& 10 x + 8 y \leq \ 80 &\text{Time constraint}\\
& 7 x + 11 y \leq \ 77 &\text{Materials constraint}\\
& x \leq 8 &\text{Capacity constraint}\\
& x, y \geq 0 &\text{Non-negativity constraints}\\
\end{align}
$$

In [4]:
# import Pkg; Pkg.add("Plots") # Uncomment and run this line if you need to install Plots package
using Plots
plotly() # use plotly backend: https://docs.juliaplots.org/latest/generated/plotly/

┌ Info: For saving to png with the Plotly backend ORCA has to be installed.
└ @ Plots /Users/jdj2/.julia/packages/Plots/M1wcx/src/backends.jl:373


Plots.PlotlyBackend()

In [8]:
# set up parameters and variables
pa = 150
pb = 175
a = range(0,15,step=0.5)
b = range(0,10,step=0.5)

# define objective function
f(a, b) = begin  (pa*a + pb*b)  end 

contour(a,b,(a,b)->f(a,b),nlevels=10, c=:heat, linewidth=10, colorbar = false, contour_labels = true)
title!("Factory Optimization Problem")
xaxis!("x=Widget A")
yaxis!("y=Widget B")
xticks!(0:maximum(a))
yticks!(0:maximum(b))
areaplot!(a[a.<=11], (77 .- 7*a[a.<=11])./11, legend=false, opacity=0.3)
areaplot!(a[a.<=8], (80 .- 10*a[a.<=8])./8, legend=false, opacity=0.3)
areaplot!(a[a.<=8], repeat([maximum(b)],length(a[a.<=8])), legend=false, opacity=0.15)
#areaplot!([8;0;4.888], [0;7;3.888], legend=false, opacity=1.0)
#scatter!([8 0 4.88],[0 7 3.888])

<!DOCTYPE html>
 
 
 Plots.jl

JuMP allows us to define an algebraic model that represents this optimization problem, which we'll do in several steps now...

### 1. Define the model

First, we **define the model**. A JuMP model object is a container for variables, constraints, solver options, etc. Models are created with the [Model()](https://jump.dev/JuMP.jl/stable/solvers/#JuMP.Model-Tuple{}) function. 

In [9]:
factory_model = Model();

### 2. Specify the optimization solver

To solve a model, we need to also specify the [optimization solver](https://jump.dev/JuMP.jl/stable/solvers/#) (or "optimizer") for JuMP to use. 

A solver is a software package that incorporates computanionally efficient algorithms for finding solutions to one or more classes of optimization problems -- e.g. [simplex](https://en.wikipedia.org/wiki/Simplex_algorithm) and [interior-point](https://en.wikipedia.org/wiki/Interior-point_method) algorithms for solving linear programming (LP) problems; [branch-and-bound](https://en.wikipedia.org/wiki/Branch_and_bound) and [branch-and-cut](https://en.wikipedia.org/wiki/Branch_and_cut) algorithms for solving mixed-integer linear programming (MILP) problems.

JuMP currently supports a number of open-source and commercial solvers which can be viewed
[here](http://www.juliaopt.org/JuMP.jl/v0.19.1/installation/#Getting-Solvers-1).

We specify the specific solver to use for this model either by using the [set_optimizer()](https://jump.dev/JuMP.jl/stable/solvers/#JuMP.set_optimizer) function or as an argument for the [Model()](https://jump.dev/JuMP.jl/stable/solvers/#JuMP.Model-Tuple{}) function when initially defining the model, as in the following examples...

In [10]:
set_optimizer(factory_model, Clp.Optimizer)
factory_model = Model(Clp.Optimizer);

Above, we specify the Clp solver interface. The argument `Clp.Optimizer` is an example of a JuMP "Optimizer Factory" parameter, a call to create an interface with a specific solver. Other examples include `GLPK.Optimizer`, `CPLEX.Optimizer`, `Gurobi.Optimizer`, etc.

### 3. Define variables

Next, we define our [decision variables](https://jump.dev/JuMP.jl/stable/variables/), using the JuMP [@variable](https://jump.dev/JuMP.jl/stable/variables/#JuMP.@variable) [macro](https://docs.julialang.org/en/v1/manual/metaprogramming/#man-macros). 

The first argument in the `@variable()` macro is a reference to the model object that this decision variable is associated with, in this case `factory_model`. 

We can also pass lower and/or upper bounds on the decision variables at the time they are created using `<=` and `>=` notation...

In [11]:
@variable(factory_model, 8 >= x >= 0) # How many widgets of type A, including capacity constraint as upper bound

x

In [12]:
@variable(factory_model, y >= 0) # How many widgets of type B

y

...Here we set a lower bound of 0 for both variables (e.g. ```y >= 0```), to reflect the non-negativity constraints in our decision variables (e.g. that we cannot produce a negative amount of widgets), and an upper bound of 8 for variable $x$, to reflect the capacity constraint on production of widget A (```8 >= x >=0```).

You can also specify bounds by using the `lower_bound` and `upper_bound` keyword arguments (as in `@variable(model, x, lower_bound = 0, upper_bound = 8`). 

A variable without bounds is considered a "free variable" and is defined by simply ommitting any bounds (as in `@variable(model, free_x)`).

The name provided for the variable in the `@variable` macro is the "[base name](https://jump.dev/JuMP.jl/stable/variables/#Variable-names-1)" assigned to this variable (e.g. `x` and `y` for our variables above). 

You can use the variable's base name as an argument in other JuMP functions to query attributes of this variable (or to recover a variable's value after solving the model). For example, we can check the upper bound on the `x` variable as follows...

In [13]:
upper_bound(x)

8.0

Note that JuMP requires unique names for variables, expressions, and constraints. If you try to define the same variable name more than once, JuMP will throw an error to prevent you from accidentally overwriting the same variable (or constraint or expression). 

In [14]:
@variable(factory_model, x >= 0) # Try defining x for a second time with different bounds

LoadError: An object of name x is already attached to this model. If this is intended, consider using the anonymous construction syntax, e.g., x = @variable(model, [1:N], ...) where the name of the object does not appear inside the macro.

You can see that the $x$ variable still has the old upper bound and has not been modified by the above attempt to define it again. 

In [15]:
upper_bound(x)

8.0

If we'd really wanted to modify the bounds for an existing JuMP variable, we would use the the `set_lower_bound` and `set_upper_bound` functions, or the ` delete_lower_bound` and `delete_upper_bound` functions to remove a previously established bound.

You can check if a name has already been assigned to a valid JuMP object (variable, expression, constraint) with the `is_valid(model_reference, JuMP_base_name)` function, and [delete an existing JuMP object](https://jump.dev/JuMP.jl/dev/variables/#Deleting-variables-1) with the `delete(model_reference, JuMP_base_name)` function.

In [18]:
is_valid(factory_model, x)

true

You can also use JuMP's `all_variables(model_reference)` function to enumerate a vector of references to all defined JuMP variables

In [19]:
all_variables(factory_model)

2-element Array{VariableRef,1}:
 x
 y

### 4. Define constraints

Next, we can define our more complicated constraints, which involve multiple variables.

We have two constraints to define in this simple example, our time constraint, $10 x + 8 y \leq \ 80$ and our material constraint, $7 x + 11 y \leq \ 77$.

We define constraints with the JuMP [@constraint](https://jump.dev/JuMP.jl/stable/constraints/#The-@constraint-macro-1) macro.

In [20]:
@constraint(factory_model, time_constraint, 10x + 8y <= 80) # Time constraint

time_constraint : 10 x + 8 y ≤ 80.0

In [22]:
@constraint(factory_model, materials_constraint, 7x + 11y <= 77) # Materials constraint

materials_constraint : 7 x + 11 y ≤ 77.0

We can now call the model object we've been building, and JuMP will helpfully print out the mathematical formulation

In [23]:
factory_model

A JuMP Model
Feasibility problem with:
Variables: 2
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Clp
Names registered in the model: materials_constraint, time_constraint, x, y

As you can see, we've now defined all of the constraints in our simple factory model. But we're missing one very important thing...

### 5. Define objective function

Currently, our model only defines a feasible region (e.g. a set of constraints over the domain of our defined decision variables). But we have not specified what our objective function is: what are we trying to maximize or minimize?

To define our objective function, we use the [@objective](https://jump.dev/JuMP.jl/dev/objective/#JuMP.@objective) macro. We have three arguments for this macro: (1) the model object we are defining the objective for, (2) specify whether this is a maximization (`Max`) or minimization (`Min`) problem, and (3) define the objective function itself.

In [25]:
@objective(factory_model, Max, 150x + 175y)

150 x + 175 y

Now we can see the full model is defined...

In [26]:
factory_model

A JuMP Model
Maximization problem with:
Variables: 2
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Clp
Names registered in the model: materials_constraint, time_constraint, x, y

### 6. Alternative ways to define our model

We can define more than one variable at a time using a more compact syntax by using the [@variables](https://jump.dev/JuMP.jl/stable/variables/#variables-1) macro. Here we'll define both x and y variables at the same time...

In [27]:
factory_model = Model(Clp.Optimizer);
@variables(factory_model, begin
    8 >= x >= 0
    y >= 0
    end)
print(factory_model)

Feasibility
Subject to
 x ≥ 0.0
 y ≥ 0.0
 x ≤ 8.0


Note that the @variables macro does not automatically print the variable bounds, so we use the `print(model_name)` function above to print out the newly defined variables and their bounds.

We can also use the [@constraints](https://jump.dev/JuMP.jl/stable/constraints/#The-@constraints-macro-1) macro in a similar manner to define multiple constraints in a more compact syntax, if preferred...

In [28]:
@constraints(factory_model, begin
    time_constraint, 10x + 8y <= 80
    materials_constraint, 7x + 11y <= 77    
    end)
print(factory_model)

Feasibility
Subject to
 time_constraint : 10 x + 8 y ≤ 80.0
 materials_constraint : 7 x + 11 y ≤ 77.0
 x ≥ 0.0
 y ≥ 0.0
 x ≤ 8.0


Sometimes it is useful to define an algebraic [expression](https://jump.dev/JuMP.jl/stable/expressions/#) that may be insert in constraints, objective functions, or other expressions (e.g. to build up more complex expressions or to define an expression used repeatedly in multiple constraints). 

The recommended way to create an expression is via the [@expression](https://jump.dev/JuMP.jl/stable/expressions/#JuMP.@expression) macro, which is syntactically similar to the definition fo a constraint. The difference is that an expression is not directly part of the optimization problem definition. It only becomes incorporated into the model definition if you subsequently use a defined expression to define part of a constraint or objective function. As an example, let's use an expression here to define our objective function...

In [29]:
@expression(factory_model, objective, 150x + 175y)
@objective(factory_model, Max, objective)
print(factory_model)

Max 150 x + 175 y
Subject to
 time_constraint : 10 x + 8 y ≤ 80.0
 materials_constraint : 7 x + 11 y ≤ 77.0
 x ≥ 0.0
 y ≥ 0.0
 x ≤ 8.0


### 7. Solve the model

Now that we've defined our factory model, let's solve it to find the optimal values $(x^*, y^*)$ that maximizes our objective function. 

To optimize a defined JuMP model for which a solver has been previously specified, use the [optimize!](https://jump.dev/JuMP.jl/stable/solvers/#JuMP.optimize!) function, supplying the model name as an argument...

In [30]:
optimize!(factory_model)

Coin0506I Presolve 2 (0) rows, 2 (0) columns and 4 (0) elements
Clp0006I 0  Obj 0 Dual inf 325 (2)
Clp0006I 2  Obj 1413.8889
Clp0000I Optimal - objective value 1413.8889
Clp0032I Optimal objective 1413.888889 - 2 iterations time 0.002


We see that the solver (Clp) solved our optimization problem and found the optimal objective function value of 1413.888889. 

### 8. Query the optimal solution

But what are the optimal decision variable values $(x^*, y^*)$? Now that the model has been solved to optimality, we can query the values of JuMP variables by using JuMP's [value()](https://jump.dev/JuMP.jl/stable/solutions/#JuMP.value) function

In [31]:
value(x)

4.888888888888888

In [32]:
value(y)

3.8888888888888897

Quick note: because JuMP uses [different containers](https://jump.dev/JuMP.jl/dev/containers/) depending on whether we've defined a single variable, a vector or square array of variables, or an array of variables with differing dimension lengths, they [recommend](https://jump.dev/JuMP.jl/dev/solutions/#Obtaining-solutions-1) using the [vectorized syntax or "dot syntax"](https://docs.julialang.org/en/v1/manual/functions/#man-vectorized) for the `value()` function, `value.(ref)`, rather than the `value(ref)` syntax used above. 

The `value.(ref)` syntax will work with any vector or array of variables and also with single variables (which can be treated as a vector of length 1), while the `value(ref)` syntax will only work when called for single variables. In this case, since we have a single variable only, we can use either syntax. We'll see an example of a vector of variables in the next section of this notebook...

In [33]:
value.(x)

4.888888888888888

In [34]:
value.(y)

3.8888888888888897

We can now see the optimal solution to our simple factory optimization model is $(x^*, y^*) = (4.8889, 3.8889)$, just as we determined in the [Constrained Optimization notebook](Notebooks/Constrained-Optimization.ipynb) via linear algebra.

In [35]:
contour(a,b,(a,b)->f(a,b),nlevels=10, c=:heat, linewidth=10, colorbar = false, contour_labels = true)
title!("Factory Optimization Problem")
xaxis!("x=Widget A")
yaxis!("y=Widget B")
xticks!(0:maximum(a))
yticks!(0:maximum(b))
areaplot!(a[a.<=11], (77 .- 7*a[a.<=11])./11, legend=false, opacity=0.3)
areaplot!(a[a.<=8], (80 .- 10*a[a.<=8])./8, legend=false, opacity=0.3)
areaplot!(a[a.<=8], repeat([maximum(b)],length(a[a.<=8])), legend=false, opacity=0.15)
scatter!([value.(x)],[value.(y)], markercolor="blue")

<!DOCTYPE html>
 
 
 Plots.jl

We can also use the `value()` function to evaluate the value of named expressions and constraints we've previously defined and associated with the solved model. (Again, vectorized dot syntax is recommended, but both forms work for single constraints).

In [36]:
value.(time_constraint)

80.0

We can see that this is the same value as we'd get by evaluating the time constraint at the optimal values $(x^*, y^*)$

In [37]:
10*value.(x) + 8*value.(y)

80.0

We can also evaluate the `objective` expression we defined previously, $150x + 175y$, and save this to a variable names `optimal_revenue` (e.g. for use later).

In [69]:
optimal_revenue = value(objective)

1413.888888888889

Or we can directly query the objective function value associated with our `factory_model` JuMP model object by using JuMP's `objective_value()` function and passing a reference to the model as follows. Since we don't always define an expression for our objective function, this is the typical way to access the objective function value for our optimal solution.

In [70]:
optimal_revenue = objective_value(factory_model)

1413.888888888889

Note that if there are certain expressions that you want to be able to quickly extract from the model's results, it can make sense to define an expression prior to solving the model, even if you do not use the expression to build a constraint or define the objective function that makes up the actual constrained optimization problem. 

For example, if we wanted to quickly extract the sum of widgets produced $(x+y)$ without individually querying each decision variable value, we could define an expression `@expression(factory_model, total_widgets, x+y)`, and then simply use `value(total_widgets)` to get this sum. 

Obviously, this is a pretty simple expression, but for more complex models, this can be a handy approach to make it easy to extract results of interest.

In [45]:
@expression(factory_model, total_widgets, x+y)

x + y

In [46]:
value.(total_widgets)

8.777777777777779

## A More Complex Model: Optimal Solar + Battery Storage Operations

The factory example above is a pretty simple problem. Useful for introducing the basic anatomy of a JuMP model, but hardly worth relying on algebraic programming to find the solution (as it can be obtained analytically without much trouble). So let's now turn to a more complex example...

<img src="img/solar.png" style="width: 800px; height: auto" align="left">

Assume you are a renewable energy company and you own a 50 megawatt (MW) utility-scale solar PV system and recently installed a 10 MW / 40 megawatt-hours (MWh) Lithium-ion battery system at the same site. Both solar and battery systems take advantage of the same DC-AC inverters (solar PV and batteries are both direct current devices) and transmission grid connection (substation).

Given a series of electricity prices, how should you operate your solar PV and storage system to maximize revenue? When do you charge, and when do you discharge? Should you buy extra power from the grid when it's cheap and store it for later when prices are high, or just make use of your solar PV system to charge the battery? What's the best operational plan to maximize revenue over the next 24 hours?

Sounds like a good optimization problem!

### 1. Load data and define parameters

Before we get started defining our model, we need to load some useful data and define a set of **parameters** or constant values used in our optimization problem.

For purposes of this example, we will use **time series data** for a single day of 15-minute [real-time electricity market prices](http://mis.ercot.com/misapp/GetReports.do?reportTypeId=13061&reportTitle=Historical%20RTM%20Load%20Zone%20and%20Hub%20Prices&showHTMLView=&mimicKey) from the Western Hub of the Electricity Reliability Council of Texas (ERCOT) electricity market, along with simulated solar PV output from [Renewables.Ninja](https://www.renewables.ninja/), which we'll load in from a .csv file with help from Julia's [CSV](https://juliadata.github.io/CSV.jl/stable/) and [DataFrames](https://juliadata.github.io/DataFrames.jl/stable/) packages.

In [123]:
# Note, you don't need to run this block if CSV and DataFrames packages are already installed
import Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [47]:
using CSV
using DataFrames
time_series = DataFrame(CSV.File("data/solar_battery_data.csv"))

,T,Hour,Interval,Price,Solar
,Int64,Int64,Int64,Float64,Float64
1,1,1,1,12.77,0.0
2,2,1,2,11.82,0.0
3,3,1,3,12.39,0.0
4,4,1,4,10.17,0.0
5,5,2,1,5.86,0.0
6,6,2,2,2.69,0.0
7,7,2,3,1.46,0.0
8,8,2,4,1.41,0.0
9,9,3,1,0.03,0.0


In [101]:
using Plots
plotly() # use plotly backend: https://docs.juliaplots.org/latest/generated/plotly/

Plots.PlotlyBackend()

In [48]:
plot(time_series.Price, label="Price (\$/MWh)")
plot!(time_series.Solar, label="Solar PV (MW-dc)")
title!("Price and Solar PV time series")
xaxis!("Time Interval (15 mins)")
yaxis!("")

<!DOCTYPE html>
 
 
 Plots.jl

Additionally, we'll define the following **capacity parameters** for each of the major subcomponents of our solar + battery system...

In [49]:
solar_cap   = 50      # Solar PV capacity in MW-dc
battery_cap = 10      # Lithium ion battery power (charge/discharge) capacity in MW-dc
energy_cap  = 40      # Lithium ion battery energy capacity (in MWh)
grid_cap    = 40      # DC-AC inverter and grid connection capacity (in MW-ac)
@show solar_cap
@show battery_cap
@show energy_cap
@show grid_cap

solar_cap = 50
battery_cap = 10
energy_cap = 40
grid_cap = 40


40

Next, we will define a set of **efficiency parameters**, which help govern the flow of electricity through our solar + storage system.

In [50]:
battery_eff = 0.95    # 'Single-trip' charge or discharge efficiency for the Lithium ion battery
inverter_eff = 0.9    # Efficiency of DC-AC inverter
@show battery_eff
@show inverter_eff

battery_eff = 0.95
inverter_eff = 0.9


0.9

Finally, we set the **starting state of charge** of the battery at the beginning of the simulation day, which we'll assume is half full.

In [51]:
start_charge = 0.5*energy_cap

20.0

### 2. Define the model

Now that we have our parameters set, let's define a new optimization model, a linear programming problem using the Clp open source solver.

In [52]:
PV_model = Model(Clp.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Clp

### 3. Define decision variables

Now we have to define our decision variables. For each of our time intervals, we have three important **operating decisions** to make:

1. How much DC power to produce (or curtail) from our solar PV system
2. How much DC power to charge the battery
3. How much DC power to discharge from the battery

For the purposes of this optimization problem, we will consider these discrete time decisions taken every 15 minutes. 

Additionally, we will define a fourth variable:

4. How much energy is stored in the battery (storage state of charge)

While this state of charge variable is technically defined as a decision variable, it is actually completely constrainted by the other variables, as we'll see below. This is an example of a [state variable](https://en.wikipedia.org/wiki/State_variable). We'll come back to state variables in dynamical systems when we introduce time coupling constraints in more depth in the Economic Dispatch notebook... 

Since we take the same kinds of decisions in each 15 minute time interval, we will thus define **an array of decision variables** for each type of decision, indexed across all time periods `t` in the set `T` as defined by our time series data `time_series.T`. 

This is our first example of creating an array of variables and **indexing across sets**. In practice, you'll frequently define arrays of variables and constraints that repeat across various sets or dimensions of your data (e.g. time periods, generators, regions, transmission lines). Note that variables can be indexed across multiple dimensions in N-dimensional containers (although here we will only index across the single dimension: time periods). [Read more on variable containers here](https://jump.dev/JuMP.jl/dev/variables/#Variable-containers-1).

Additionally, we will define bounds for each decision variable based on the capacity parameters set previously. We'll set solar's upper bounds later, as these vary by time interval depending on the availability of solar insolation at the time.

Note: by convention, and to make it easy to spot a decision variable in future expressions, we'll use the convention that all decision variable names will follow the format `UPPERCASE`. (This is an optional convention, and you can feel free to adopt your own, but we recommend adopting a consistent convention for naming variables, constraints, expressions, parameters, etc.)

In [53]:
# Decision variables and bounds:

@variables(PV_model, begin
        battery_cap    >= CHARGE[t in time_series.T]    >= 0
        battery_cap    >= DISCHARGE[t in time_series.T] >= 0
        energy_cap     >= SOC[t in time_series.T]       >= 0
                          SOLAR[t in time_series.T]     >= 0
end)
PV_model

A JuMP Model
Feasibility problem with:
Variables: 384
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 384 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 288 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Clp
Names registered in the model: CHARGE, DISCHARGE, SOC, SOLAR

### 4. Define expressions

Now we define an  expressions that relate the flow of electricity within our solar + storage system. 

Note that by convention, and to distinguish expression names from decision variable names and constraint names, we will use the convention that all expression names take the format `eExpressionName`. (This is an optional convention, and you can feel free to adopt your own, but we recommend adopting a consistent convention for naming variables, constraints, expressions, parameters, etc.)

1. First, the **export** (if positive) or **import** (if negative) of power to the grid can be expressed by the sum of solar PV and battery discharge less battery charging in each time period `t` accounting for battery and inverter efficiencies. 

$\begin{align*} 
\quad eExportImport_t = inverter\_eff (SOLAR_t + DISCHARGE_t - CHARGE_t) \quad \forall t \in T
\end{align*}$

As with our variables, we will define this expression as an array indexed across the set of time periods `t` in the time series `T`, since this holds for all time periods...

In [54]:
# Grid export/import flow. Positive domain values = exports to grid, negative domain values = imports from grid
@expression(PV_model, eExportImport[t in time_series.T], inverter_eff*(SOLAR[t] + DISCHARGE[t] - CHARGE[t]))

1-dimensional DenseAxisArray{GenericAffExpr{Float64,VariableRef},1,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  87, 88, 89, 90, 91, 92, 93, 94, 95, 96]
And data, a 96-element Array{GenericAffExpr{Float64,VariableRef},1}:
 0.9 SOLAR[1] + 0.9 DISCHARGE[1] - 0.9 CHARGE[1]
 0.9 SOLAR[2] + 0.9 DISCHARGE[2] - 0.9 CHARGE[2]
 0.9 SOLAR[3] + 0.9 DISCHARGE[3] - 0.9 CHARGE[3]
 0.9 SOLAR[4] + 0.9 DISCHARGE[4] - 0.9 CHARGE[4]
 0.9 SOLAR[5] + 0.9 DISCHARGE[5] - 0.9 CHARGE[5]
 0.9 SOLAR[6] + 0.9 DISCHARGE[6] - 0.9 CHARGE[6]
 0.9 SOLAR[7] + 0.9 DISCHARGE[7] - 0.9 CHARGE[7]
 0.9 SOLAR[8] + 0.9 DISCHARGE[8] - 0.9 CHARGE[8]
 0.9 SOLAR[9] + 0.9 DISCHARGE[9] - 0.9 CHARGE[9]
 0.9 SOLAR[10] + 0.9 DISCHARGE[10] - 0.9 CHARGE[10]
 0.9 SOLAR[11] + 0.9 DISCHARGE[11] - 0.9 CHARGE[11]
 0.9 SOLAR[12] + 0.9 DISCHARGE[12] - 0.9 CHARGE[12]
 0.9 SOLAR[13] + 0.9 DISCHARGE[13] - 0.9 CHARGE[13]
 ⋮
 0.9 SOLAR[85] + 0.9 DISCHARGE[85] - 0.9 CHARGE[85]
 0.9 SOLAR[86] + 0.9 DISCHARGE[86] - 0.9 CHARG

Note that JuMP automatically interprets our expression definition `inverter_eff*(SOLAR[t] + DISCHARGE[t] - CHARGE[t])` by converting the parameter variable references to constants and distributing terms into an affine (linear) expression, as per the displayed text above. So feel free to define these expressions however you'd like, and JuMP will take care of the math...

2. Second, the **total revenue** from sale or purchase of electricity in each time period is expressed by the sum product of prices in each interval and the net flow of power in each interval. Since prices are expressed per whole MWh and our time intervals are 15 minutes, we multiply the flow of power by 0.25 to determine the total energy sold/purchased.

$\begin{align*} 
\quad eRevenue = \sum_{t \in T} eExportImport_t \times Price_t \times 0.25  = \sum_{t \in T}inverter\_eff (SOLAR_t + DISCHARGE_t - CHARGE_t) \times Price_t \times 0.25
\end{align*}$

Here we see our first example of the syntax for a sum expression.

In [55]:
@expression(PV_model, eRevenue, sum(eExportImport[t]*time_series.Price[t]*0.25 for t in time_series.T))

2.87325 SOLAR[1] + 2.87325 DISCHARGE[1] - 2.87325 CHARGE[1] + 2.6595 SOLAR[2] + 2.6595 DISCHARGE[2] - 2.6595 CHARGE[2] + 2.7877500000000004 SOLAR[3] + 2.7877500000000004 DISCHARGE[3] - 2.7877500000000004 CHARGE[3] + 2.28825 SOLAR[4] + 2.28825 DISCHARGE[4] - 2.28825 CHARGE[4] + 1.3185 SOLAR[5] + 1.3185 DISCHARGE[5] - 1.3185 CHARGE[5] + 0.60525 SOLAR[6] + 0.60525 DISCHARGE[6] - 0.60525 CHARGE[6] + 0.3285 SOLAR[7] + 0.3285 DISCHARGE[7] - 0.3285 CHARGE[7] + 0.31725 SOLAR[8] + 0.31725 DISCHARGE[8] - 0.31725 CHARGE[8] + 0.00675 SOLAR[9] + 0.00675 DISCHARGE[9] - 0.00675 CHARGE[9] - 0.04275 SOLAR[10] - 0.04275 DISCHARGE[10] + 0.04275 CHARGE[10] - 0.162 SOLAR[11] - 0.162 DISCHARGE[11] + 0.162 CHARGE[11] - 0.12150000000000001 SOLAR[12] - 0.12150000000000001 DISCHARGE[12] + 0.12150000000000001 CHARGE[12] - 0.10125 SOLAR[13] - 0.10125 DISCHARGE[13] + 0.10125 CHARGE[13] - 0.135 SOLAR[14] - 0.135 DISCHARGE[14] + 0.135 CHARGE[14] - 0.162 SOLAR[15] - 0.162 DISCHARGE[15] + 0.162 CHARGE[15] - 0.14175000

### 5. Constraints

Now we define a set of constraints that define the feasible region for our decision variables, based on the engineering constraints for our solar and storage system.

Note that by convention, and to distinguish constraint names from decision variable names and expression names, we will use the convention that all constraint names take the format `cExpressionName`. (This is an optional convention, and you can feel free to adopt your own, but we recommend adopting a consistent convention for naming variables, constraints, expressions, parameters, etc.)

1. First, power imported or exported to/from the grid cannot be greater than the grid connection capacity. We'll make use of our previously defined expression for net exports/imports here, and since the expression can take on positive values (indicating exports) and negative values (indicating imports), we will define upper and lower bound values here on this expression equal to the grid capacity parameter.

$\begin{align*} 
\quad grid\_cap \geq eExportImport_t \geq -grid\_cap  \quad \forall t \in T
\end{align*}$

In [56]:
@constraint(PV_model, 
    cGridCap[t in time_series.T], # Named array of constraints indexed across all times t in T
    grid_cap >= eExportImport[t] >= -grid_cap  # Constraint definition
)

1-dimensional DenseAxisArray{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.Interval{Float64}},ScalarShape},1,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  87, 88, 89, 90, 91, 92, 93, 94, 95, 96]
And data, a 96-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.Interval{Float64}},ScalarShape},1}:
 cGridCap[1] : 0.9 SOLAR[1] + 0.9 DISCHARGE[1] - 0.9 CHARGE[1] ∈ [-40.0, 40.0]
 cGridCap[2] : 0.9 SOLAR[2] + 0.9 DISCHARGE[2] - 0.9 CHARGE[2] ∈ [-40.0, 40.0]
 cGridCap[3] : 0.9 SOLAR[3] + 0.9 DISCHARGE[3] - 0.9 CHARGE[3] ∈ [-40.0, 40.0]
 cGridCap[4] : 0.9 SOLAR[4] + 0.9 DISCHARGE[4] - 0.9 CHARGE[4] ∈ [-40.0, 40.0]
 cGridCap[5] : 0.9 SOLAR[5] + 0.9 DISCHARGE[5] - 0.9 CHARGE[5] ∈ [-40.0, 40.0]
 cGridCap[6] : 0.9 SOLAR[6] + 0.9 DISCHARGE[6] - 0.9 CHARGE[6] ∈ [-40.0, 40.0]
 cGridCap[7] : 0.9 SOLAR[7] + 0.9 DISCHARGE[7] - 0

You can see that JuMP automatically interprets the expressions here to construct constraints that are in terms of decision variables only. 

2. The **state of charge** of the battery is constrainted to be equal to the state of charge in the prior time period `t-1` plus the energy charged during this time period less the energy discharged during this time period, accounting for storage efficiency losses. 

$\begin{align*} 
\quad SOC_t = SOC_{t-1} + 0.25\big(CHARGE_t * battery\_eff - \frac{DISCHARGE_t}{battery\_eff}\big)  \quad \forall t \in T
\end{align*}$

Note that as we model 15 minute time steps here, we multiply the charge and discharge power (in MW) by 0.25 to determine the change in storage energy (in MWh). 

This is an example of a recursive constraint: the state of charge in each period depends on the state of charge in the prior period. So we have to define a special case for the first time period, with the starting state of charge as the simulation defined by a parameter earlier. As each expression refers to the expression in the previous time step, we will proceed through a `for` loop to define each expression...

In [57]:
# First define an Array of length equal to our time series to contain references to each expression
cStateOfCharge = Array{Any}(undef, length(time_series.T))

# First period state of charge:
cStateOfCharge[1] = @constraint(PV_model, 
    SOC[1] == start_charge + (CHARGE[1]*battery_eff - DISCHARGE[1]/battery_eff)*0.25
) 

# All other time periods, defined recursively based on prior state of charge
for t in time_series.T[(time_series.T .> 1)]
    cStateOfCharge[t] = @constraint(PV_model, 
        SOC[t] == SOC[t-1] + CHARGE[t]*battery_eff*0.25 - DISCHARGE[t]/battery_eff*0.25
    )
end

cStateOfCharge # display constraints

96-element Array{Any,1}:
 SOC[1] - 0.2375 CHARGE[1] + 0.2631578947368421 DISCHARGE[1] = 20.0
 SOC[2] - SOC[1] - 0.2375 CHARGE[2] + 0.2631578947368421 DISCHARGE[2] = 0.0
 SOC[3] - SOC[2] - 0.2375 CHARGE[3] + 0.2631578947368421 DISCHARGE[3] = 0.0
 SOC[4] - SOC[3] - 0.2375 CHARGE[4] + 0.2631578947368421 DISCHARGE[4] = 0.0
 SOC[5] - SOC[4] - 0.2375 CHARGE[5] + 0.2631578947368421 DISCHARGE[5] = 0.0
 SOC[6] - SOC[5] - 0.2375 CHARGE[6] + 0.2631578947368421 DISCHARGE[6] = 0.0
 SOC[7] - SOC[6] - 0.2375 CHARGE[7] + 0.2631578947368421 DISCHARGE[7] = 0.0
 SOC[8] - SOC[7] - 0.2375 CHARGE[8] + 0.2631578947368421 DISCHARGE[8] = 0.0
 SOC[9] - SOC[8] - 0.2375 CHARGE[9] + 0.2631578947368421 DISCHARGE[9] = 0.0
 SOC[10] - SOC[9] - 0.2375 CHARGE[10] + 0.2631578947368421 DISCHARGE[10] = 0.0
 SOC[11] - SOC[10] - 0.2375 CHARGE[11] + 0.2631578947368421 DISCHARGE[11] = 0.0
 SOC[12] - SOC[11] - 0.2375 CHARGE[12] + 0.2631578947368421 DISCHARGE[12] = 0.0
 SOC[13] - SOC[12] - 0.2375 CHARGE[13] + 0.2631578947368421 

3. Finally, we must constrain the **maximum solar PV output** to reflect the weather-dependent variation in solar insolation in each time period. The maximum available solar power output in each time interval is provided by the parameters in the Solar time series input data. Thus, the constraint is of the form:

$\begin{align*} 
SOLAR \leq time\_series.Solar_t \quad \forall t \in T
\end{align*}$

In [58]:
@constraint(PV_model, cMaxSolar[t in time_series.T], 
    SOLAR[t] <= time_series.Solar[t]
)

1-dimensional DenseAxisArray{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  87, 88, 89, 90, 91, 92, 93, 94, 95, 96]
And data, a 96-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 cMaxSolar[1] : SOLAR[1] ≤ 0.0
 cMaxSolar[2] : SOLAR[2] ≤ 0.0
 cMaxSolar[3] : SOLAR[3] ≤ 0.0
 cMaxSolar[4] : SOLAR[4] ≤ 0.0
 cMaxSolar[5] : SOLAR[5] ≤ 0.0
 cMaxSolar[6] : SOLAR[6] ≤ 0.0
 cMaxSolar[7] : SOLAR[7] ≤ 0.0
 cMaxSolar[8] : SOLAR[8] ≤ 0.0
 cMaxSolar[9] : SOLAR[9] ≤ 0.0
 cMaxSolar[10] : SOLAR[10] ≤ 0.0
 cMaxSolar[11] : SOLAR[11] ≤ 0.0
 cMaxSolar[12] : SOLAR[12] ≤ 0.0
 cMaxSolar[13] : SOLAR[13] ≤ 0.0
 ⋮
 cMaxSolar[85] : SOLAR[85] ≤ 0.0
 cMaxSolar[86] : SOLAR[86] ≤ 0.0
 cMaxSolar[87] : SOLAR[87] ≤ 0.0
 cMaxSolar[

### 6. Set Objective Function

Finally, we set our objective function, which is to maximize the revenue from our solar and storage system across the time series (one day). We can make use of the revenue expression we defined previous:

$\begin{align*} 
\max eRevenue = \sum_{t \in T} eExportImport_t \times Price_t \times 0.25 = \sum_{t \in T} inverter\_eff (SOLAR_t + DISCHARGE_t - CHARGE_t) \times Price_t \times 0.25
\end{align*}$

In [59]:
@objective(PV_model, Max, eRevenue)

2.87325 SOLAR[1] + 2.87325 DISCHARGE[1] - 2.87325 CHARGE[1] + 2.6595 SOLAR[2] + 2.6595 DISCHARGE[2] - 2.6595 CHARGE[2] + 2.7877500000000004 SOLAR[3] + 2.7877500000000004 DISCHARGE[3] - 2.7877500000000004 CHARGE[3] + 2.28825 SOLAR[4] + 2.28825 DISCHARGE[4] - 2.28825 CHARGE[4] + 1.3185 SOLAR[5] + 1.3185 DISCHARGE[5] - 1.3185 CHARGE[5] + 0.60525 SOLAR[6] + 0.60525 DISCHARGE[6] - 0.60525 CHARGE[6] + 0.3285 SOLAR[7] + 0.3285 DISCHARGE[7] - 0.3285 CHARGE[7] + 0.31725 SOLAR[8] + 0.31725 DISCHARGE[8] - 0.31725 CHARGE[8] + 0.00675 SOLAR[9] + 0.00675 DISCHARGE[9] - 0.00675 CHARGE[9] - 0.04275 SOLAR[10] - 0.04275 DISCHARGE[10] + 0.04275 CHARGE[10] - 0.162 SOLAR[11] - 0.162 DISCHARGE[11] + 0.162 CHARGE[11] - 0.12150000000000001 SOLAR[12] - 0.12150000000000001 DISCHARGE[12] + 0.12150000000000001 CHARGE[12] - 0.10125 SOLAR[13] - 0.10125 DISCHARGE[13] + 0.10125 CHARGE[13] - 0.135 SOLAR[14] - 0.135 DISCHARGE[14] + 0.135 CHARGE[14] - 0.162 SOLAR[15] - 0.162 DISCHARGE[15] + 0.162 CHARGE[15] - 0.14175000




Let's take a look at our full model...


In [60]:
PV_model

A JuMP Model
Maximization problem with:
Variables: 384
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 96 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 96 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.Interval{Float64}`: 96 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 384 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 288 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Clp
Names registered in the model: CHARGE, DISCHARGE, SOC, SOLAR, cGridCap, cMaxSolar, eExportImport, eRevenue


A little more complicated, eh? Now you can see why it is useful to have an algebraic programming language to define more complex models and then hand over to a solver to do the hard math. 

As JuMP enumerates each individual variable when printing the model, here's a more compact algebraic definition of the solar and storage optimal dispatch problem:


$$
\begin{align}
& \max \sum_{t \in T} inverter\_eff (SOLAR_t + DISCHARGE_t - CHARGE_t) \times Price_t \times 0.25 && \text{Objective function (revenue)}\\[6pt] 
& \text{s.t.} && \\[6pt]
& SOC_t = SOC_{t-1} + 0.25 \big(CHARGE_t * battery\_eff - \frac{DISCHARGE_t}{battery\_eff}\big)  &\forall t \in T \text{ & } t > 1 & \\[6pt]
& SOC_1 = start\_charge + 0.25 \big(CHARGE_1 * battery\_eff - \frac{DISCHARGE_1}{battery\_eff}\big)  & &\quad \text{State of charge constraints}\\[6pt]
& grid\_cap \geq inverter\_eff (SOLAR_t + DISCHARGE_t - CHARGE_t)  \geq -grid\_cap  &\forall t \in T & \quad \text{Grid capacity constraints}\\[6pt]
& CHARGE_t     \leq battery\_cap         &\forall t \in T & \quad \text{Charge capacity constraints}\\[6pt]
& DISCHARGE_t    \leq battery\_cap      &\forall t \in T & \quad \text{Discharge capacity constraints}\\[6pt]
& SOC_t    \leq energy\_cap             &\forall t \in T & \quad \text{State of charge capacity constraints}\\[6pt]
& SOLAR_t  \leq time\_series.Solar_t     &\forall t \in T & \quad \text{Solar PV capacity constraints}\\[6pt]
& CHARGE_t, DISCHARGE_t, SOC_t, SOLAR_t \geq 0 &\forall t \in T & \quad \text{Non-negativity constraints}\\[6pt]
\end{align}
$$




Let's otimize this problem!

In [61]:
optimize!(PV_model)

Coin0506I Presolve 119 (-169) rows, 310 (-74) columns and 458 (-309) elements
Clp0006I 0  Obj 877.28539 Primal inf 41.039133 (1) Dual inf 685.07167 (126)
Clp0006I 73  Obj 8071.3562 Primal inf 346.33526 (66)
Clp0006I 113  Obj 8071.3562 Primal inf 2788.8354 (71)
Clp0006I 167  Obj 7294.8605 Primal inf 13.426733 (8)
Clp0006I 175  Obj 7290.0468
Clp0000I Optimal - objective value 7290.0468
Coin0511I After Postsolve, objective 7290.0468, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 7290.04677 - 175 iterations time 0.002, Presolve 0.00


### Results!

We can see from the solver that the optimal revenue is \$7920. 

We can now access the optimal decision variables as well. As these are now arrays of variables, we must use the [vectorized syntax](https://docs.julialang.org/en/v1/manual/functions/#man-vectorized) version of JuMP's `value()` function: `value.(ArrayOfVariablesReference)` 

Similarly, for or arrays of expressions or constraints, use: `value.(ArrayOfExpressionsReference)` or `value.(ArrayOfConstraintsReference)`.

For example, here's the optimal state of charge for the battery across the day:

In [62]:
value.(SOC)

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  87, 88, 89, 90, 91, 92, 93, 94, 95, 96]
And data, a 96-element Array{Float64,1}:
 17.36842105263158
 14.736842105263158
 12.105263157894736
  9.473684210526315
  6.842105263157896
  4.210526315789475
  2.0000000000000115
  2.0000000000000115
  4.3750000000000115
  6.7500000000000115
  9.125000000000012
 11.50000000000001
 13.87500000000001
  ⋮
  0.0
  0.5131578947368411
  2.888157894736841
  5.263157894736842
  2.631578947368421
  0.0
  0.0
  0.0
  0.0
  2.375
  0.0
  0.0

Additionally, we can compile all of the results into a DataFrame and write to a csv to save our outputs for further analysis.

Note however that `value.(Reference)` will return a JuMP [DenseAxisArray](https://jump.dev/JuMP.jl/dev/variables/#variable_jump_arrays-1) which is not a data format that is compatible with DataFrames. To access the numerical values for an array of JuMP variables or expressions in a DenseAxisArray, you use the syntax `value.(Reference).data`.

In [63]:
typeof(value.(SOC))

JuMP.Containers.DenseAxisArray{Float64,1,Tuple{Array{Int64,1}},Tuple{Dict{Int64,Int64}}}

In [64]:
typeof(value.(SOC).data)

Array{Float64,1}

Below we compile the DataFrame and write it to .csv file. Note that we copy in the time intervals and price in each time period from the input time series as well for context.

In [65]:
results = DataFrame(
    T = time_series.T,
    Price = time_series.Price,
    Solar_Output = value.(SOLAR).data, 
    Storage_Discharge = value.(DISCHARGE).data,
    Storage_Charge = value.(CHARGE).data,
    Storage_SOC = value.(SOC).data,
    Export_Import = value.(eExportImport).data,
    Interval_Revenue = value.(eExportImport).data.*time_series.Price
    )

,T,Price,Solar_Output,Storage_Discharge,Storage_Charge,Storage_SOC,Export_Import
,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,12.77,0.0,10.0,0.0,17.3684,9.0
2,2,11.82,0.0,10.0,0.0,14.7368,9.0
3,3,12.39,0.0,10.0,0.0,12.1053,9.0
4,4,10.17,0.0,10.0,0.0,9.47368,9.0
5,5,5.86,0.0,10.0,0.0,6.84211,9.0
6,6,2.69,0.0,10.0,0.0,4.21053,9.0
7,7,1.46,0.0,8.4,0.0,2.0,7.56
8,8,1.41,0.0,0.0,0.0,2.0,0.0
9,9,0.03,0.0,0.0,10.0,4.375,-9.0


In [66]:
# Let's create a folder for our results
path = "results" # Replace this with a different path if you prefer to write to another location
if !ispath(path) 
    mkdir(path)
end
write_path = joinpath(path, "solar_storage_results.csv") 
CSV.write(write_path, results)

"results/solar_storage_results.csv"

Finally, let's plot the optimal operational decisions for our solar and storage system...

In [67]:
plot(results.Solar_Output, label="Solar (MW-dc)")
plot!(results.Storage_Discharge, label="Discharge (MW-dc)")
plot!(-1*results.Storage_Charge, label="Charge (MW-dc)")
plot!(results.Storage_SOC, label="State Of Charge (MWh)")
title!("Optimal Solar and Storage Operations")
xaxis!("Time Interval (15 mins)")
yaxis!("")

<!DOCTYPE html>
 
 
 Plots.jl

In [68]:
plot(results.Price, label="Price (\$/MWh)")
plot!(results.Export_Import, label="Net Grid Export (MW-ac)")
title!("Optimal Net Grid Export and Price")
xaxis!("Time Interval (15 mins)")
yaxis!("")

<!DOCTYPE html>
 
 
 Plots.jl